# 🧠 SENTALIS — Rule-Based Sentiment Analysis
## + Export Dataset untuk Machine Learning

**Notebook ini mencakup:**
1. Load & preview data komentar Instagram
2. Text preprocessing (cleaning, normalisasi)
3. Rule-based sentiment classification (Indonesia + Sunda)
4. Visualisasi hasil analisis
5. **Export dataset berlabel siap pakai untuk training ML**

---
> 📁 Letakkan file `komentar_instagram.csv` di direktori yang sama dengan notebook ini,
> atau upload ke Google Colab terlebih dahulu.

## 📦 Cell 1 — Install & Import Library

In [ ]:
# Install library yang dibutuhkan
!pip install pandas matplotlib seaborn wordcloud openpyxl -q

import pandas as pd
import numpy as np
import re
import json
import os
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter

# Agar grafik tampil di notebook
%matplotlib inline
plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.family'] = 'DejaVu Sans'

print('✅ Semua library berhasil diimport!')

## 📂 Cell 2 — Load Data

In [ ]:
# ─── Ganti path jika file ada di lokasi lain ───
CSV_PATH = 'komentar_instagram.csv'

# Auto-detect encoding
for enc in ['utf-8-sig', 'utf-8', 'latin-1']:
    try:
        df = pd.read_csv(CSV_PATH, encoding=enc)
        print(f'✅ File berhasil dibaca dengan encoding: {enc}')
        break
    except Exception:
        continue

# Normalisasi nama kolom
df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]

# Pastikan kolom 'komentar' ada
if 'komentar' not in df.columns:
    raise ValueError(f'Kolom "komentar" tidak ditemukan. Kolom yang ada: {list(df.columns)}')

# Hapus baris tanpa teks komentar
df = df.dropna(subset=['komentar']).reset_index(drop=True)
df['komentar'] = df['komentar'].astype(str).str.strip()
df = df[df['komentar'].str.len() > 0].reset_index(drop=True)

print(f'\n📊 Total data: {len(df)} baris')
print(f'📋 Kolom: {list(df.columns)}')
print(f'\n--- Preview 5 baris pertama ---')
display(df.head())

## 🧹 Cell 3 — Text Preprocessing

In [ ]:
def preprocess(text):
    """
    Membersihkan dan menormalisasi teks komentar.
    - Lowercase
    - Hapus URL, mention, hashtag
    - Hapus karakter non-ASCII yang tidak perlu
    - Normalisasi spasi berlebih
    """
    if not isinstance(text, str):
        return ''
    text = text.lower().strip()
    text = re.sub(r'http\S+|www\.\S+', '', text)          # hapus URL
    text = re.sub(r'@\w+', '', text)                        # hapus mention
    text = re.sub(r'#\w+', '', text)                        # hapus hashtag
    text = re.sub(r'[^\w\s.,!?\'-]', ' ', text)            # hapus karakter aneh
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)             # kurangi repetisi (hahaha -> haha)
    text = re.sub(r'\s+', ' ', text).strip()               # normalisasi spasi
    return text

df['teks_bersih'] = df['komentar'].apply(preprocess)

# Tampilkan perbandingan sebelum & sesudah
contoh = df[['komentar', 'teks_bersih']].head(8)
print('🔍 Contoh hasil preprocessing:\n')
for _, row in contoh.iterrows():
    print(f'  Asli  : {row["komentar"][:80]}')
    print(f'  Bersih: {row["teks_bersih"][:80]}')
    print()

## 🗂️ Cell 4 — Kamus Kata & Frasa Sentimen

> Kamus ini bisa diedit sesuai kebutuhan. Semakin lengkap kamus, semakin akurat hasilnya.

In [ ]:
# ═══════════════════════════════════════════════════════════════
#  KAMUS FRASA NEGATIF
#  (dicek lebih dulu untuk menangkap sarkasme & frasa khusus)
# ═══════════════════════════════════════════════════════════════
FRASA_NEGATIF = [
    # Sarkasme eksplisit
    'puas mreun', 'alhamdulillah katanya', 'katanya sudah', 'sok bagus',
    'wkwk', 'wkwkwk', 'haha', 'hahaha', 'bisa ketawa',
    'alhamdulillah masih', 'alhamdulillah belum', 'alhamdulillah ga',
    'alhamdulillah tdk', 'alhamdulillah tidak', 'alhamdulillah blm',
    # Kritik infrastruktur
    'jalan rusak', 'jalan berlubang', 'jalan parah', 'jalan ancur',
    'jalan hancur', 'jalan mana', 'jalan boro', 'jalan barutut',
    'masih rusak', 'masih berlubang', 'masih gelap', 'masih banjir',
    'masih macet', 'masih kumuh', 'masih kotor',
    # Kritik pemerintah
    'tidak ada perubahan', 'ga ada perubahan', 'gak ada perubahan',
    'teu ngaruh', 'teu aya', 'teu robah', 'teu berubah',
    'ga ngaruh', 'gak ngaruh', 'tidak ngaruh',
    'ga ada', 'gak ada', 'tidak ada hasil',
    'kitu keneh', 'kitu wae', 'kitu deui', 'kitu bae',
    'php', 'gimmick', 'pencitraan', 'omong kosong', 'janji palsu',
    'bohong', 'tidak terbukti', 'ga terbukti', 'gak terbukti',
    'tidak nyata', 'tidak berhasil', 'ga berhasil',
    # Ungkapan kekecewaan
    'kecewa', 'mengecewakan', 'nyebelin', 'menyebalkan', 'parah',
    'gila', 'kurang ajar', 'carut marut', 'ancur', 'hancur lebur',
    'amburadul', 'berantakan', 'tidak beres', 'ga beres',
    'kapan selesai', 'kapan beres', 'kapan beneran',
    # Sunda negatif
    'butut', 'paroek', 'awon', 'goreng pisan', 'teu bener',
    'teu puguh', 'teu jelas', 'teu karuhan', 'teu nanaon',
    'ngan omongan', 'ukur omongan', 'omong wungkul',
    # Keluhan spesifik Sumedang
    'galian', 'galian pasir', 'tambang', 'debu', 'bising',
    'banjir terus', 'macet terus', 'gelap terus', 'kumuh terus',
    'pengangguran', 'susah kerja', 'sulitnya kerja', 'tidak ada kerjaan',
    'ga ada kerjaan', 'lapangan kerja kurang',
]

# ═══════════════════════════════════════════════════════════════
#  KATA NEGATIF (kata tunggal)
# ═══════════════════════════════════════════════════════════════
KATA_NEGATIF = [
    'jelek', 'buruk', 'rusak', 'gagal', 'lambat', 'lambat', 'lemot',
    'kotor', 'bau', 'kumuh', 'jorok', 'semrawut', 'kacau', 'amburadul',
    'tidak', 'belum', 'kurang', 'minim', 'nihil', 'nol',
    'susah', 'sulit', 'payah', 'repot', 'ribet', 'pelik',
    'benci', 'marah', 'kesel', 'kesal', 'sebal', 'gondok',
    'bohong', 'dusta', 'tipu', 'menipu', 'nipu',
    'korupsi', 'korup', 'pungli', 'sogok', 'suap',
    'diskriminasi', 'nepotisme', 'kolusi',
    'banjir', 'longsor', 'macet', 'sempit', 'gelap',
    'mahal', 'mencekik', 'mencekam', 'memberatkan',
    # Sunda
    'butut', 'awon', 'gorengan', 'teu', 'euweuh',
]

# ═══════════════════════════════════════════════════════════════
#  FRASA POSITIF
# ═══════════════════════════════════════════════════════════════
FRASA_POSITIF = [
    'terima kasih', 'terimakasih', 'makasih', 'thank you',
    'alhamdulillah ada perubahan', 'ada kemajuan', 'ada perbaikan',
    'sudah lebih baik', 'sudah bagus', 'sudah maju', 'semakin bagus',
    'semakin baik', 'semakin maju', 'semakin berkembang',
    'sukses selalu', 'semoga sukses', 'semoga berhasil',
    'sehat selalu', 'semoga sehat', 'lancar terus',
    'terus berjuang', 'terus maju', 'terus berkembang',
    'kerja bagus', 'kerja keras', 'kerja nyata', 'kerja baik',
    'mantap jiwa', 'top banget', 'luar biasa', 'keren banget',
    'hebat banget', 'salut', 'salut pisan',
    # Sunda positif
    'alhamdulillah aya', 'alus pisan', 'sae pisan', 'sae teuing',
    'hatur nuhun', 'nuhun', 'sukur',
]

# ═══════════════════════════════════════════════════════════════
#  KATA POSITIF (kata tunggal)
# ═══════════════════════════════════════════════════════════════
KATA_POSITIF = [
    'bagus', 'baik', 'mantap', 'keren', 'hebat', 'luar biasa',
    'sukses', 'berhasil', 'maju', 'berkembang', 'meningkat',
    'bersih', 'rapi', 'indah', 'nyaman', 'aman', 'sejuk',
    'ramah', 'sopan', 'profesional', 'cepat', 'tepat',
    'transparan', 'jujur', 'amanah', 'adil',
    'inovatif', 'kreatif', 'solutif', 'responsif',
    'senang', 'bahagia', 'puas', 'bangga', 'lega', 'syukur',
    'terima', 'apresiasi', 'dukung', 'mendukung',
    # Sunda
    'alus', 'sae', 'saé',
]

# Urutkan dari terpanjang ke terpendek (penting untuk matching akurat)
FRASA_NEGATIF.sort(key=len, reverse=True)
FRASA_POSITIF.sort(key=len, reverse=True)

print(f'📖 Kamus siap:')
print(f'   Frasa negatif : {len(FRASA_NEGATIF)} entri')
print(f'   Kata negatif  : {len(KATA_NEGATIF)} entri')
print(f'   Frasa positif : {len(FRASA_POSITIF)} entri')
print(f'   Kata positif  : {len(KATA_POSITIF)} entri')

## 🔍 Cell 5 — Fungsi Klasifikasi Sentimen

In [ ]:
def klasifikasi_sentimen(teks):
    """
    Mengklasifikasikan sentimen teks menggunakan pendekatan rule-based.
    
    Urutan pemeriksaan (hierarki):
    1. Frasa negatif (untuk tangkap sarkasme)
    2. Frasa positif
    3. Kata negatif
    4. Kata positif
    5. Default: Netral
    
    Returns:
        tuple: (label: str, kata_kunci: list, skor: int)
    """
    if not teks or not isinstance(teks, str) or len(teks.strip()) < 2:
        return 'Netral', [], 0
    
    t = teks.lower()
    cocok_negatif = []
    cocok_positif = []
    
    # 1. Cek frasa negatif
    for frasa in FRASA_NEGATIF:
        if frasa in t:
            cocok_negatif.append(frasa)
    
    # 2. Cek frasa positif
    for frasa in FRASA_POSITIF:
        if frasa in t:
            cocok_positif.append(frasa)
    
    # 3. Cek kata negatif (hanya jika belum ada frasa negatif)
    if not cocok_negatif:
        for kata in KATA_NEGATIF:
            if re.search(r'\b' + re.escape(kata) + r'\b', t):
                cocok_negatif.append(kata)
    
    # 4. Cek kata positif (hanya jika belum ada frasa positif)
    if not cocok_positif:
        for kata in KATA_POSITIF:
            if re.search(r'\b' + re.escape(kata) + r'\b', t):
                cocok_positif.append(kata)
    
    # 5. Tentukan label berdasarkan jumlah match
    skor_neg = len(cocok_negatif)
    skor_pos = len(cocok_positif)
    skor_net = skor_pos - skor_neg  # positif = nilai tinggi
    
    if skor_neg > skor_pos:
        return 'Negatif', cocok_negatif[:5], skor_net
    elif skor_pos > skor_neg:
        return 'Positif', cocok_positif[:5], skor_net
    elif skor_neg > 0 and skor_neg == skor_pos:
        # Jika sama banyak, frasa negatif lebih prioritas
        return 'Negatif', cocok_negatif[:5], skor_net
    else:
        return 'Netral', [], 0


# ── Test fungsi ──
test_cases = [
    'Ga ngaruh apa2. hahaha',
    'Jalan barutut jeng paroek, kapan diperbaiki?',
    'Alhamdulillah ada perubahan terutama di fasilitas umum',
    'Sukses selalu bapa bupati!',
    'Hmmmm',
    'Survey na wak',
    'Gimmick doang, PHP warga!',
]

print('🧪 Test klasifikasi:\n')
for t in test_cases:
    bersih = preprocess(t)
    label, kunci, skor = klasifikasi_sentimen(bersih)
    print(f'  [{label:8s}] skor={skor:+2d} | "{t[:55]}"')
    if kunci:
        print(f'           kata kunci: {kunci}')
    print()

## ⚙️ Cell 6 — Jalankan Klasifikasi ke Seluruh Data

In [ ]:
# Terapkan klasifikasi ke semua baris
hasil = df['teks_bersih'].apply(klasifikasi_sentimen)

df['sentimen']    = hasil.apply(lambda x: x[0])   # label sentimen
df['kata_kunci']  = hasil.apply(lambda x: ', '.join(x[1]) if x[1] else '')  # kata yang cocok
df['skor']        = hasil.apply(lambda x: x[2])   # skor numerik

# Ringkasan distribusi
dist = df['sentimen'].value_counts()
total = len(df)

print('📊 Hasil Klasifikasi:\n')
for label, count in dist.items():
    pct = count / total * 100
    bar = '█' * int(pct / 3)
    print(f'  {label:10s} {bar:20s} {count:4d} ({pct:.1f}%)')

print(f'\n  Total: {total} komentar')
print('\n─── Preview 10 hasil ───')
display(df[['komentar', 'sentimen', 'skor', 'kata_kunci']].head(10))

## 📊 Cell 7 — Visualisasi Distribusi Sentimen

In [ ]:
WARNA = {'Negatif': '#FF3B30', 'Netral': '#8E8E93', 'Positif': '#34C759'}

dist_sorted = df['sentimen'].value_counts()
labels = dist_sorted.index.tolist()
counts = dist_sorted.values.tolist()
colors = [WARNA.get(l, '#007AFF') for l in labels]

fig, axes = plt.subplots(1, 2, figsize=(13, 5))
fig.patch.set_facecolor('#F2F2F7')

# ── Pie chart ──
ax1 = axes[0]
ax1.set_facecolor('#F2F2F7')
wedges, texts, autotexts = ax1.pie(
    counts, labels=labels, colors=colors,
    autopct='%1.1f%%', startangle=140,
    wedgeprops=dict(edgecolor='white', linewidth=2.5),
    pctdistance=0.75
)
for at in autotexts:
    at.set_fontsize(11)
    at.set_fontweight('bold')
    at.set_color('white')
for t in texts:
    t.set_fontsize(12)
ax1.set_title('Distribusi Sentimen', fontsize=14, fontweight='bold', pad=15, color='#1C1C1E')

# ── Bar chart ──
ax2 = axes[1]
ax2.set_facecolor('white')
bars = ax2.barh(labels, counts, color=colors, edgecolor='white', height=0.55)
ax2.set_xlabel('Jumlah Komentar', fontsize=11, color='#3A3A3C')
ax2.set_title('Jumlah per Kategori', fontsize=14, fontweight='bold', color='#1C1C1E')
ax2.tick_params(colors='#3A3A3C')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
for bar, count in zip(bars, counts):
    ax2.text(count + 1, bar.get_y() + bar.get_height() / 2,
             f'{count}', va='center', fontsize=11, fontweight='bold', color='#1C1C1E')

plt.tight_layout(pad=2)
plt.savefig('viz_distribusi.png', dpi=150, bbox_inches='tight', facecolor='#F2F2F7')
plt.show()
print('💾 Tersimpan: viz_distribusi.png')

## 📊 Cell 8 — Visualisasi per Tipe (Komentar vs Reply)

In [ ]:
if 'tipe' in df.columns:
    cross = pd.crosstab(df['tipe'], df['sentimen'])
    for col in ['Negatif', 'Netral', 'Positif']:
        if col not in cross.columns:
            cross[col] = 0
    cross = cross[['Negatif', 'Netral', 'Positif']]

    fig, ax = plt.subplots(figsize=(9, 5))
    fig.patch.set_facecolor('#F2F2F7')
    ax.set_facecolor('white')

    x = np.arange(len(cross.index))
    w = 0.25
    for i, (col, color) in enumerate(WARNA.items()):
        if col in cross.columns:
            bars = ax.bar(x + i * w, cross[col], w, label=col, color=color, edgecolor='white')
            for bar in bars:
                h = bar.get_height()
                if h > 0:
                    ax.text(bar.get_x() + bar.get_width() / 2, h + 0.5,
                            str(int(h)), ha='center', va='bottom', fontsize=9, fontweight='bold')

    ax.set_xticks(x + w)
    ax.set_xticklabels(cross.index, fontsize=11)
    ax.set_ylabel('Jumlah', fontsize=11)
    ax.set_title('Sentimen berdasarkan Tipe Komentar', fontsize=13, fontweight='bold', color='#1C1C1E')
    ax.legend(title='Sentimen', fontsize=10)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.tight_layout()
    plt.savefig('viz_tipe.png', dpi=150, bbox_inches='tight', facecolor='#F2F2F7')
    plt.show()
    print('💾 Tersimpan: viz_tipe.png')
else:
    print('ℹ️ Kolom "tipe" tidak ditemukan, visualisasi ini dilewati.')

## ☁️ Cell 9 — WordCloud per Sentimen

In [ ]:
# Stopwords gabungan Indonesia + Sunda
STOPWORDS = set([
    'yang', 'dan', 'di', 'ke', 'dari', 'ini', 'itu', 'ada', 'dengan',
    'untuk', 'pada', 'akan', 'adalah', 'juga', 'bisa', 'nya', 'saja',
    'sudah', 'belum', 'lebih', 'agar', 'atau', 'karena', 'jika', 'bila',
    'kita', 'kami', 'saya', 'kamu', 'dia', 'mereka', 'anda', 'bapak',
    'ibu', 'pak', 'bu', 'pa', 'min', 'kak', 'bang',
    # Sunda
    'mah', 'teh', 'wae', 'oge', 'kitu', 'kieu', 'nu', 'ka', 'na',
    'we', 'eta', 'atuh', 'yeuh', 'euy', 'lah', 'sih', 'ih', 'ah',
    'naha', 'kumaha', 'naon', 'iraha', 'dimana', 'keneh', 'pisan',
    # Umum chat
    'banget', 'bgt', 'aja', 'ajah', 'dong', 'deh', 'yah', 'ya', 'yg',
    'jd', 'tp', 'dgn', 'utk', 'krn', 'pd', 'sm', 'dr', 'skrg', 'sdh',
])

sentimen_labels = ['Negatif', 'Netral', 'Positif']
warna_wc = {'Negatif': 'Reds', 'Netral': 'Greys', 'Positif': 'Greens'}

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.patch.set_facecolor('#F2F2F7')

for ax, label in zip(axes, sentimen_labels):
    subset = df[df['sentimen'] == label]['teks_bersih']
    teks_gabung = ' '.join(subset.tolist())

    if len(teks_gabung.strip()) < 10:
        ax.text(0.5, 0.5, f'Data {label}\nterlalu sedikit',
                ha='center', va='center', fontsize=12, transform=ax.transAxes)
        ax.axis('off')
        continue

    wc = WordCloud(
        width=500, height=350,
        background_color='white',
        colormap=warna_wc[label],
        stopwords=STOPWORDS,
        max_words=60,
        min_font_size=10,
        prefer_horizontal=0.85,
        collocations=False
    ).generate(teks_gabung)

    ax.imshow(wc, interpolation='bilinear')
    ax.axis('off')
    ax.set_title(f'WordCloud — {label}\n({len(subset)} komentar)',
                 fontsize=12, fontweight='bold',
                 color=list(WARNA.values())[sentimen_labels.index(label)])

plt.tight_layout(pad=1.5)
plt.savefig('viz_wordcloud.png', dpi=150, bbox_inches='tight', facecolor='#F2F2F7')
plt.show()
print('💾 Tersimpan: viz_wordcloud.png')

## 🔢 Cell 10 — Kata Paling Sering Muncul per Sentimen

In [ ]:
def top_kata(df_subset, n=12):
    semua_kata = []
    for teks in df_subset['teks_bersih']:
        kata_list = [k for k in teks.split() if k not in STOPWORDS and len(k) > 2]
        semua_kata.extend(kata_list)
    return Counter(semua_kata).most_common(n)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.patch.set_facecolor('#F2F2F7')

for ax, label in zip(axes, sentimen_labels):
    subset = df[df['sentimen'] == label]
    top = top_kata(subset)
    if not top:
        ax.text(0.5, 0.5, 'Tidak ada data', ha='center', va='center', transform=ax.transAxes)
        ax.axis('off')
        continue

    kata_list, freq_list = zip(*top)
    color = WARNA[label]
    ax.set_facecolor('white')
    bars = ax.barh(list(reversed(kata_list)), list(reversed(freq_list)),
                   color=color, alpha=0.85, edgecolor='white')
    ax.set_title(f'Top Kata — {label}', fontsize=12, fontweight='bold', color=color)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='y', labelsize=9)
    ax.set_xlabel('Frekuensi', fontsize=9)

plt.suptitle('Kata Paling Sering Muncul per Sentimen', fontsize=14, fontweight='bold',
             color='#1C1C1E', y=1.01)
plt.tight_layout()
plt.savefig('viz_topkata.png', dpi=150, bbox_inches='tight', facecolor='#F2F2F7')
plt.show()
print('💾 Tersimpan: viz_topkata.png')

## 💾 Cell 11 — Export Dataset untuk Machine Learning

> **Ini bagian utama!** Dataset yang dihasilkan di sini siap digunakan untuk:
> - Training model Machine Learning (Naive Bayes, SVM, BERT, dll.)
> - Fine-tuning LLM
> - Transfer learning / dataset benchmark
>
> Format yang tersedia: **CSV**, **Excel**, **JSON**, **JSONL** (Hugging Face format)

In [ ]:
# ═══════════════════════════════════════════════════════════
#  BUAT DATASET ML
#  Setiap baris = satu sampel data berlabel
# ═══════════════════════════════════════════════════════════

# Label encoding untuk ML
LABEL_MAP = {'Negatif': 0, 'Netral': 1, 'Positif': 2}

# Kolom yang dibutuhkan untuk ML
kolom_ml = {
    'id': range(1, len(df) + 1),
    'teks_asli':   df['komentar'].values,
    'teks_bersih': df['teks_bersih'].values,
    'label':       df['sentimen'].values,             # label string
    'label_id':    df['sentimen'].map(LABEL_MAP).values,  # label angka (0/1/2)
    'skor':        df['skor'].values,
    'kata_kunci':  df['kata_kunci'].values,
}

# Tambahkan kolom opsional jika ada
if 'username' in df.columns:
    kolom_ml['username'] = df['username'].fillna('').values
if 'waktu' in df.columns:
    kolom_ml['waktu'] = df['waktu'].fillna('').values
if 'tipe' in df.columns:
    kolom_ml['tipe'] = df['tipe'].fillna('').values
if 'likes' in df.columns:
    kolom_ml['likes'] = pd.to_numeric(df['likes'], errors='coerce').fillna(0).astype(int).values

df_ml = pd.DataFrame(kolom_ml)

print('📦 Dataset ML siap:')
print(f'   Baris   : {len(df_ml)}')
print(f'   Kolom   : {list(df_ml.columns)}')
print(f'   Label   : {df_ml["label"].value_counts().to_dict()}')
print(f'   Label ID: {dict(LABEL_MAP)}')
print()
display(df_ml[['id', 'teks_asli', 'teks_bersih', 'label', 'label_id', 'skor']].head(10))

In [ ]:
# ─── Export ke berbagai format ───

# 1. CSV (format paling umum untuk ML)
df_ml.to_csv('dataset_ml.csv', index=False, encoding='utf-8-sig')
print('✅ Tersimpan: dataset_ml.csv')

# 2. Excel (untuk review manual)
with pd.ExcelWriter('dataset_ml.xlsx', engine='openpyxl') as writer:
    df_ml.to_excel(writer, sheet_name='Dataset ML', index=False)
    # Sheet ringkasan
    ringkasan = pd.DataFrame({
        'Label': list(LABEL_MAP.keys()),
        'Label ID': list(LABEL_MAP.values()),
        'Jumlah': [len(df_ml[df_ml['label'] == l]) for l in LABEL_MAP.keys()],
        'Persentase (%)': [round(len(df_ml[df_ml['label'] == l]) / len(df_ml) * 100, 2)
                           for l in LABEL_MAP.keys()],
    })
    ringkasan.to_excel(writer, sheet_name='Ringkasan', index=False)
print('✅ Tersimpan: dataset_ml.xlsx')

# 3. JSON (untuk API / Python dict)
df_ml.to_json('dataset_ml.json', orient='records', force_ascii=False, indent=2)
print('✅ Tersimpan: dataset_ml.json')

# 4. JSONL — format Hugging Face / OpenAI fine-tuning
with open('dataset_ml.jsonl', 'w', encoding='utf-8') as f:
    for _, row in df_ml.iterrows():
        record = {
            'text':     row['teks_bersih'],
            'label':    row['label'],
            'label_id': int(row['label_id']),
        }
        f.write(json.dumps(record, ensure_ascii=False) + '\n')
print('✅ Tersimpan: dataset_ml.jsonl (Hugging Face format)')

# 5. Split train/test (80/20) — siap untuk training
from sklearn.model_selection import train_test_split  # pip install scikit-learn jika belum ada

try:
    df_train, df_test = train_test_split(
        df_ml, test_size=0.2, random_state=42,
        stratify=df_ml['label']  # proporsional tiap kelas
    )
    df_train.to_csv('dataset_train.csv', index=False, encoding='utf-8-sig')
    df_test.to_csv('dataset_test.csv',  index=False, encoding='utf-8-sig')
    print(f'✅ Train: {len(df_train)} baris → dataset_train.csv')
    print(f'✅ Test : {len(df_test)} baris  → dataset_test.csv')
    print(f'   Distribusi train: {df_train["label"].value_counts().to_dict()}')
    print(f'   Distribusi test : {df_test["label"].value_counts().to_dict()}')
except ImportError:
    print('⚠️  scikit-learn belum terinstall. Jalankan: !pip install scikit-learn')
    print('   Train/test split dilewati.')

print('\n🎉 Export selesai!')

## 📋 Cell 12 — Metadata & Dokumentasi Dataset

> File `dataset_metadata.json` ini penting agar orang lain (atau model ML kita) tahu
> konteks dan cara membaca dataset ini.

In [ ]:
# Hitung statistik teks
df_ml['panjang_teks'] = df_ml['teks_bersih'].str.len()
df_ml['jumlah_kata']  = df_ml['teks_bersih'].str.split().str.len()

metadata = {
    'nama_dataset': 'SENTALIS — Instagram Comment Sentiment Dataset',
    'sumber': '@inimahsumedang (Instagram)',
    'konteks': 'Komentar postingan evaluasi setahun kepemimpinan Bupati Sumedang',
    'bahasa': ['Indonesia', 'Sunda', 'campuran'],
    'metode_labeling': 'Rule-based (keyword + frasa, hierarki negatif → positif → netral)',
    'catatan': 'Label bersifat estimasi otomatis, belum diverifikasi manusia penuh',
    'statistik': {
        'total_sampel': int(len(df_ml)),
        'distribusi_label': df_ml['label'].value_counts().to_dict(),
        'distribusi_label_id': df_ml['label_id'].value_counts().to_dict(),
        'rata_panjang_teks': round(df_ml['panjang_teks'].mean(), 1),
        'rata_jumlah_kata': round(df_ml['jumlah_kata'].mean(), 1),
        'teks_terpendek': int(df_ml['panjang_teks'].min()),
        'teks_terpanjang': int(df_ml['panjang_teks'].max()),
    },
    'skema_label': {
        '0 = Negatif': 'Komentar mengandung kritik, kekecewaan, atau sentimen buruk',
        '1 = Netral':  'Komentar faktual, ambigu, atau tidak mengandung muatan emosi jelas',
        '2 = Positif': 'Komentar mengandung apresiasi, dukungan, atau sentimen baik',
    },
    'kolom_ml_utama': {
        'teks_bersih': 'Input untuk model (sudah di-preprocess)',
        'label':       'Target label string (Negatif/Netral/Positif)',
        'label_id':    'Target label angka (0/1/2) untuk ML',
    },
    'file_tersedia': [
        'dataset_ml.csv       — dataset lengkap',
        'dataset_ml.xlsx      — dataset + sheet ringkasan',
        'dataset_ml.json      — format JSON records',
        'dataset_ml.jsonl     — format Hugging Face/OpenAI',
        'dataset_train.csv    — 80% untuk training',
        'dataset_test.csv     — 20% untuk evaluasi',
    ],
    'rekomendasi_ml_selanjutnya': [
        'TF-IDF + Naive Bayes (baseline cepat)',
        'TF-IDF + SVM (akurasi lebih baik)',
        'IndoBERT fine-tuning (terbaik untuk bahasa Indonesia)',
        'Pertimbangkan human annotation untuk meningkatkan kualitas label',
    ]
}

with open('dataset_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print('✅ Tersimpan: dataset_metadata.json')
print()
print('📋 Ringkasan Metadata:')
print(f'   Sumber         : {metadata["sumber"]}')
print(f'   Total sampel   : {metadata["statistik"]["total_sampel"]}')
print(f'   Rata kata/teks : {metadata["statistik"]["rata_jumlah_kata"]} kata')
print(f'   Distribusi     : {metadata["statistik"]["distribusi_label"]}')
print()
print('─── Skema Label ───')
for k, v in metadata['skema_label'].items():
    print(f'   {k}: {v}')

## 📥 Cell 13 — Download Semua File (Google Colab)

In [ ]:
import zipfile

# Kumpulkan semua file output
files_to_zip = [
    'dataset_ml.csv',
    'dataset_ml.xlsx',
    'dataset_ml.json',
    'dataset_ml.jsonl',
    'dataset_train.csv',
    'dataset_test.csv',
    'dataset_metadata.json',
    'viz_distribusi.png',
    'viz_wordcloud.png',
    'viz_topkata.png',
]
if os.path.exists('viz_tipe.png'):
    files_to_zip.append('viz_tipe.png')

zip_name = 'SENTALIS_output.zip'
with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zf:
    for fname in files_to_zip:
        if os.path.exists(fname):
            zf.write(fname)
            print(f'  + {fname}')
        else:
            print(f'  - {fname} (tidak ditemukan, dilewati)')

print(f'\n📦 ZIP berhasil dibuat: {zip_name}')
print(f'   Ukuran: {os.path.getsize(zip_name) / 1024:.1f} KB')
print()

# Auto-download jika di Google Colab
try:
    from google.colab import files
    files.download(zip_name)
    print('⬇️  Download dimulai...')
except ImportError:
    print(f'📂 File tersimpan di direktori saat ini: {os.path.abspath(zip_name)}')
    print('   (Jika di Colab, jalankan ulang cell ini setelah semua cell selesai)')

---

## 🗺️ Langkah Selanjutnya — dari Rule-Based ke Machine Learning

Dataset yang sudah diexport bisa langsung digunakan untuk:

| Pendekatan | Library | File Input |
|---|---|---|
| **TF-IDF + Naive Bayes** | `sklearn` | `dataset_train.csv` + `dataset_test.csv` |
| **TF-IDF + SVM** | `sklearn` | `dataset_train.csv` + `dataset_test.csv` |
| **IndoBERT / BERT** | `transformers` | `dataset_ml.jsonl` |
| **GPT fine-tuning** | `openai` | `dataset_ml.jsonl` |

**Rekomendasi:** Lakukan **human review** minimal pada komentar yang `skor` mendekati 0  
(ambiguous) sebelum training model untuk hasil yang lebih akurat.

```python
# Filter komentar ambigu untuk review manual:
df_ambigu = df_ml[df_ml['skor'] == 0]
df_ambigu.to_csv('review_manual.csv', index=False)
```

---
*SENTALIS — Sentiment Analysis Intelligent System*  
*Rule-Based Edition dengan ML Export*